![Logo 1](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech1.jpg)
<div class="alert alert-block alert-info">
<h1> Praktyczne zastosowania chmury obliczeniowej </h1>
<h2> 3.3. <i>Infrastructure as a Service - Hetzner</i>  [laboratoria]</h2> 
<h3> Patryk Żywica (2021)</h3>
</div>

![Logo 2](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech2.jpg)

# Zadania do realizacji w czasie zajęć

## Założenie konta na Hetzner Cloud

1. Wejdź na stronę <https://accounts.hetzner.com> i zarejestruj nowe konto. Możesz skorzystaj ze studenckiego adresu email, ułatwi to dodanie odpowiednich uprawnień do Twojego konta.

> Możesz też skorzystać z programu poleceń dzięki czemu uzyskasz 20 EUR kredytów do wykorzystania w chmurze na dowolne cele: <https://hetzner.cloud/?ref=U4w5MqYdJ9pK>

  Proces rejestracji wymaga odebrania linku aktywacyjnego oraz podania podstawowych danych osobowych. Jako formę płatności można wybrać przelew bankowy. 
  
1. Zweryfikuj dostęp do projektu o nazwie ***PZC***, którego właścicielem jest prowadzący zajęcia. Za zasoby utworzone w ramach tego projektu nie musisz płacić.

> Stare zasoby będą regularnie usuwane. Niezależnie od tego proszę usuwać wszystkie utworzone przez siebie zasoby jak tylko nie są już potrzebne. Projekt ***PZC*** ma przypisane limity na maksymalną liczbę zasobów, zostawiając je nie usunięte blokujesz możliwość pracy innym uczestnikom zajeć. Dokładne wartości limitów:
> * liczba serwerów: 19
> * liczba migawek: 30
> * wolumeny dyskowe: 1TB
> * równoważenie obciążenia: 20

> Jeśli nie chcesz stracić swoich postępów prac, a jednocześnie planujesz dłuższą przerwę możesz wyłączyć maszynę wirtualną, zrobić jej migawkę a następnie maszynę usunąć. Gdy chcesz wrócić do pracy, serwer można łatwo przywrócić na podstawie migawki. W ten sposób zmniejszasz liczbę serwerów, która jest najbardziej istotnym ograniczeniem.

## Instalacja Wordpress w środowisku Hetzner

Celem tego zadania będzie instalacja *Wordpressa* - najpopularniejszego systemu zarządzania treścią (CMS), w infrastrukturze typu IaaS z wykorzystaniem dostępu przeglądarkowego. Proces instalacji wykonamy ręcznie krok po kroku. Na następnych zajęciach przedstawione zostanie, jak zautomatyzować instalację poprzez uruchomienie skryptu.

### Centralna baza danych

Pierwszym krokiem przy instalacji *Wordpresa* jest zainstalowanie i skonfigurowanie bazy danych. Hetzner nie oferuje zarządzanych baz danych w modelu SaaS (patrz następne zajęcia), dlatego na tym etapie będziemy również odpowiedzialni za instalację bazy i jej późniejsze utrzymanie. Jednak, aby istotnie uprościć proces i wyabstrahować zbędne szczegóły, do uruchomienia oprogramowania wykorzystamy narzędzie **Docker**. 

Docelowo dążymy do uzyskania następującej konfiguracji:

![Diagram przedstawia pojedynczą maszynę wirtualną z uruchomionymi na niej dwoma kontenerami docker: bazą danych MySql oraz webowym interfejsem administracyjnym phpMyAdmin. Komunikacja pomiędzy kontenerami odbywa się za pośrednictwem wewnętrznej sieci zarządzanej przez dockera. PhpMyAdmin jest udostępniony na publicznym adresie IP maszyny wirtualnej na porcie 8080.](obrazy/wp-etap1.png)


1. Rozpoczynając pracę w interfejsem webowym [chmury obliczeniowej Hetzner](https://cloud.hetzner.com) nie dysponujemy żadnymi zasobami. W ramach zasobów dostępnych w ramach zajęć powinieneś być przydzielony do dwóch projektów: *Default* oraz *PZC*. Pamiętaj, że maszyny wirtualne utworzone w projekcie *PZC* opłacane są przez Wydział, podczas gdy za te utworzone w projekcie *Default* rachunek zostanie wystawiony na dane podane przy rejestracji konta.
  <img src="obrazy/hetzner-pusty.png" width="75%" alt="">

1. Wybieramy opcję utworzenia nowej maszyny wirtualnej i wybieramy następujące dane:

    <img src="obrazy/hetzner-vm-1.png" width="75%" alt="">
    <img src="obrazy/hetzner-vm-2.png" width="75%" alt="">
      - lokalizacja: Helsinki (nowsze serwery, lepszy transfer do polski, niższe ceny, wsparcie w języku angielskim)
      - obraz systemu: Ubuntu 20.04 lub nowsze (Ubuntu serwer jest najlepszym wyborem dla początkujących administratorów)
      - typ instancji: CX11 (1 vCPU, 2GB RAM, 20GB SSD, cena na poziomie 3 groszy za godzinę)
      - dyski: brak
  
1. W kroku *Sieć* tworzymy nową sięć prywatną o nazwie `network-1` i puli adresów `10.10.10.0/24`.
    
    <img src="obrazy/hetzner-network.png" width="75%" alt="">

1. Pomijamy dodanie zapory sieciowej oraz dodatkowych funkcjonalności. Podajemy natomiast klucz SSH, który będzie służył do logowania się na naszą maszynę na konto `root`.

    <img src="obrazy/hetzner-ssh-key.png" width="75%" alt="">

    > **Uwaga**: Teoretycznie Hetzner umożliwia logowanie się hasłem do utworzonych maszyn wirtualnych, jest to jednak bardzo nie rozważne. Proponuję przeprowadzić eksperyment polegający na utworzeniu maszyny wirtualnej z hasłem `admin` i zmierzeniu czasu po jakim dostęp do niej zostanie przejęty przez boty. Na szczęście zawsze możesz usunąć maszynę w interfejsie webowym Hetzner, nawet po przejęciu nad nią kontroli przez kogoś innego.

1. Potwierdzamy chęć utworzenia maszyny wirtualnej i czekamy na jej inicjalizację. Całość powinna trwać około minuty. Po tym czasie nasza maszyna jest gotowa do użycia. Sprawdź jej dostępność logując się przez SSH na przypisany nowoutworzonej maszynie adres IP. 

    <img src="obrazy/hetzner-po-vm.png" width="75%" alt="">

Teraz gdy dysponujemy nową maszyną wirtualną możemy rozpocząć instalację niezbędnego oprogramowania.

1. Na początku musimy zainstalować oprogramowanie docker. Niestety wersje oferowane w repozytoriach Ubuntu są bardzo stare i nie jest zalecane ich używanie. Poniższy skrypt instaluje zarówno `docker`a jak i `docker-compose` zgodnie z zaleceniami twórców tych narzędzi.
    ```bash
    apt update && apt install apt-transport-https ca-certificates curl gnupg lsb-release
    
    curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
    
    echo \
      "deb [arch=amd64 signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu \
      $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
    
    apt update && apt install docker-ce docker-ce-cli containerd.io

    sudo curl -L "https://github.com/docker/compose/releases/download/1.29.2/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
    
    chmod +x /usr/local/bin/docker-compose
    ```

1. Następnie należy stworzyć plik o nazwie `docker-compose.yml` o następującej treści. Lokalizacja pliku nie ma znaczenia.
    ```docker-compose
    version: '3.9'

    services:
      db:
        image: mysql:5.7
        restart: always
        environment:
          MYSQL_ROOT_PASSWORD: notSecureChangeMe
        volumes:
          - db_data:/var/lib/mysql

      phpmyadmin:
        image: phpmyadmin
        restart: always
        ports:
          - 8080:80
    volumes:
      db_data: {}
    ```

    W pliku tym definiujemy dwie usługi (samą bazę danych oraz webowy interfejs dostępowy). Wskazujemy też, że dostęp do bazy przez przeglądarkę będzie możliwy z portu 8080.

1. Bazę danych uruchamiamy komendą:
    ```bash
    docker-compose up
    ```
    Aby wyłączyć bazę danych należy wcisnąć `Ctrl-C`.

    > **Wskazówka**: Jeśli chcesz usunąć utworzone kontenery wraz z zasobami dyskowymi zastosuj komendę `docker-compose down -v`. W ten sposób możesz zacząć od zera.

    > **Wskazówka**: Polecenie `docker-compose up -d` uruchamia nasze kontenery w tle.

1. Odwiedź w przeglądarce adres: `http://<adres IP maszyny wirtualnej db>:8080` u zaloguj się używając nazwy użytkownika `root` oraz hasła podanego w pliku `docker-compose.yml`. Jest to webowy interfejs dostępowy do naszej bazy danych, który przyda się w dalszej części zajęć.

    <img src="obrazy/phpMyAdmin.png" width="75%" alt="">


Udało się nam skonfigurować instancję bazy MySql. W obecnej konfiguracji nie jest ona zbytnio zabezpieczona, jednak zajmiemy się tym przy okazji instalacji Wordpress w następnym kroku.

### Instalacja Wordpress

Teraz naszym zadaniem jest zainstalować oprogramowanie Wordpress na osobnej maszynie wirtualnej. Ponownie wykorzystamy gotowe obrazy `docker` w celu otrzymania następującej konfiguracji:

![Diagram przedstawia dwie maszyny wirtualne. Konfiguracja maszyny db została zaktualizowana tak aby baza danych była dostępna w sieci prywatnej. Nowa maszyna wirtualna wordpress-1 udostępnia CMS na porcie 8000 i komunikuje się z bazą danych wewnątrz sieci prywatnej.](obrazy/wp-etap2.png)

1. Na maszynie wirtualnej z bazą danych (`db`) musimy dokonać pewnych zmian w konfiguracji. Przede wszystkim konieczne jest udostępnienie bazy danych na zewnątrz. Nie chcemy jednak, aby była ona dostępna publicznie, a tylko w sieci prywatnej. Stąd podajemy konkretny prywatny adres IP przy udostępnianiu portów. Dodajemy też osobne konto i bazę danych dla Wordpresa. Wynikowy plik powinien wyglądać następująco.

    > **Uwaga**: Zwróć uwagę na przydzielony maszynie `db` prywatny adres IP, z reguły będzie to `10.10.10.2`. Możesz to sprawdzić w widoku szczegółów maszyny wirtualnej w interfejsie webowy w zakładce *Sieć*.

    ```docker-compose
    version: '3.9'

    services:
      db:
        image: mysql:5.7
        restart: always
        ports:
          - "10.10.10.2:3306:3306"
        environment:
          MYSQL_ROOT_PASSWORD: notSecureChangeMe
          MYSQL_DATABASE: wordpress
          MYSQL_USER: wordpress
          MYSQL_PASSWORD: wordpress
        volumes:
          - db_data:/var/lib/mysql

      phpmyadmin:
        image: phpmyadmin
        restart: always
        # przekierowanie portów zostawione tylko dla przykładu, nie należy tak robić na produkcji
        ports:
          - "8080:80"
    volumes:
      db_data: {}
    ```

1. Tworzymy drugą maszynę wirtualną o nazwie `wordpress-1`, wybierając identyczne opcje jak poprzednio.

1. Logujemy się przez SSH na nową maszynę wirtualną i podobnie jak poprzednio tworzymy plik `docker-compose.yml` o następującej treści.

    ```docker-compose
    version: '3.9'

    services:
      wordpress:
        image: wordpress:latest
        volumes:
          - wordpress_data:/var/www/html
        ports:
          - "8000:80"
        restart: always
        environment:
          WORDPRESS_DB_HOST: "10.10.10.2:3306"
          WORDPRESS_DB_USER: wordpress
          WORDPRESS_DB_PASSWORD: wordpress
          WORDPRESS_DB_NAME: wordpress
          
    volumes:
      wordpress_data: {}
    ```

    Tym razem korzystamy z oficjalnego obrazu Wordpress. Za pomocą zmiennych środowiskowych podajemy dane dostępowe do bazy danych. Zwróć uwagę na zmienną środowiskową `WORDPRESS_DB_HOST`, która wskazuje na adres prywatny serwera `db`.

1. W przeglądarce odwiedź adres `http://<adres IP maszyny wirtualnej wordpress-1>:8000` i dokończ instalację Wordpress.

W ten sposób zakończyła się instalacja Wordpress. Serwer web oraz baza danych znajdują się na osobnych maszynach wirtualnych, ponadto baza danych dostępna jest tylko z poziomu wirtualnej sieci prywatnej.

### Mechanizm równoważenia obciążenia i migawki systemu

W kolejnym etapie dodamy mechanizm równoważenia obciążenia oraz drugą maszynę wirtualną na której działał będzie Wordpress. W tym celu wykorzystamy mechanizm migawek maszyn wirtualnych. Dążymy teraz do otrzymania następującej konfiguracji:

![Diagram przedstawia ewolucję poprzedniej konfiguracji. Dodano mechanizm równoważenia obciążenia, który teraz nasłuchuje na porcie 80 i przekazuje ruch do dwóch niezależnych maszyn wirtualnych korzystając z sieci wewnętrznej.](obrazy/wp-etap3.png)

1. Wyłącz maszynę wirtualną `wordpress-1`. Możesz to zrobić np. wydając komendę:
    ```bash
    shutdown now
    ```
    
    Zweryfikuj, że maszyna jest wyłączona w interfejsie webowym.

1. Utwórz migawkę tej maszyny. W tym celu wejdź w widok szczegółów maszyny i wybierz opcję *Migawka*. Potwierdź chęć jej utworzenia i poczekaj aż proces dobiegnie końca.

1. Migawka pozwala na stworzenie nowej maszyny wirtualnej na bazie obecnie istniejącej. Pozwoli nam to oszczędzić czas i zapewni jednolitość obu instancji. Przejdź do tworzenia nowej maszyny wirtualnej i tym razem w sekcji *Obraz systemu* wybierz opcję *Migawka*. Pozostałe opcje ustaw tak jak poprzednio, a maszynę nazwij `wordpress-2`. W między czasie możesz też uruchomić ponownie maszynę `wordpress-1`.

1. Możesz teraz zweryfikować, że na obu maszynach działa ten sam Wordpress. Dodaj przykładowy post na jednym z nich i sprawdź, czy pojawi się na drugim.

1. Czas na stworzenie i konfigurację równoważenia obciążenia. Przechodzimy do odpowiednich opcji w interfejsie webowym i wybieramy następującą konfigurację:
    - lokalizacja: Helsinki
    - typ: LB11
    - sieć: network-1
    - cele: wybieramy obie maszyny wirtualne z Wordpresem i zaznaczamy komunikacje przez prywatny adres
    - usługi: http, port źródłowy 800, port docelowy 8000, 
    - sprawdzanie stanu maszyn: wybieramy opcję *sticky sessions* z czasem ustawionym na 60 sekund, zmieniamy port docelowy na 8000, upewniamy się że *proxy protocol* jest wyłączony

    Potwierdzamy chęć utworzenia mechanizmu równoważenia obciążenia. Jego inicjalizacja może trochę potrwać, ale w przeglądarce już można przejść pod `http://<adres IP LB>:8000/`

    <img src="obrazy/hetzner-lb.png" width="75%" alt="">

1. Zwróć uwagę, że przy próbie zalogowanie zostajemy automatycznie przekierowani na adres IP pierwszej maszyny wirtualnej. Wynika to z domyślnego zachowania Wordpresa. Aby to naprawić należy, wejść w interfejs webowy bazy danych i zmodyfikować dwa wiersze w tabeli `wp-options`: `siteurl` oraz `home`, podając tam adres IP mechanizmu równoważenia obciążenia.

    > **Uwaga**: Wordpress nie jest przystosowany do łatwego przenoszenia/zmieniania adresu URL na którym pracuje.

    > **Uwaga**: W rzeczywistych warunkach nasza strona dostępna byłaby za pomocą adresu domenowego i powinna zostać od samego początku skonfigurowana jako dostępna pod tym adresem. Adres ten powinien być następnie przekierowany na początku na maszynę `wordpres-1` a następnie na mechanizm równoważenia obciążenia. Niestety opóźnienia propagacji wpisów DNS uniemożliwiają przeprowadzenie tego w ramach zajęć. 

1. Sprawdźmy czy nasza strona jest teraz rzeczywiście obsługiwana jednocześnie przez dwie maszyny wirtualne. W tym celu korzystając z adresu IP mechanizmu równoważenia obciążenia, dodaj nowy post zawierający jeden obraz (wgrany na serwer, nie przez URL czy z biblioteki). Odczekaj chwilę (60 sekund) lub otwórz stronę w innej przeglądarce/trybie incognito. Czasem trzeba to powtórzyć kilka razy. Co się dzieje?

    > **Wskazówka**: Można też uruchomić narzędzia deweloperskie przeglądarki (Crtl-I) i ręcznie usunąć ciasteczko `HCLBSTICKY`, które powoduje utrzymanie powiązania przeglądarki z konkretną maszyną wirtualną.

1. Zaobserwowane zjawisko pokazuje, kluczowy problem z taką instalacją Wordpress. Konieczne jest utrzymanie synchronizacji pomiędzy dyskami obu maszyn wirtualnych (to tam przechowywane są przesłane na serwer obrazy). Przykładowo jeszcze większe problemy moglibyśmy obserwować przy aktualizacji Wordpresa, gdy tylko jedna z instancji zostałaby zaktualizowana. Rozwiązanie tego problemu wymaga ingerencji w Wordpresa i zostanie pokazane na jednych z następnych zajęć.

## Problemy otwarte

Zastanów się nad następującymi problemami:

1. W jaki sposób moglibyśmy zabezpieczyć się przed utratą zgormadzonych na stronie danych.
1. Jaki jest koszt takiej infrastruktury (wliczając kopie zapasowe, migawki, itp). Porównaj go z kosztem zakupu hostowanego Wordpresa w modelu SaaS na <https://www.wordpress.com>.
1. Co się stanie, gdy zmienimy skórkę w tak skonfigurowanym Wordpresie.
1. Jakie jeszcze problemy mogą wyniknąć przy takiej instalacji Wordpresa.